In [1]:
import pandas as pd
import numpy as np

In [20]:
LOW = 0.08
HIGH = 0.85

In [21]:
#load prediction file for both detection and classification
det = pd.read_csv('../checkpoint/1703/vfnet/1703_vfnet_submission.csv')
cls = pd.read_csv('../checkpoint/1703/classifier/default.csv')

In [22]:
print(det.shape)
det.head()

(3000, 2)


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.33 1139 631 1443 937 3 0.73 835 1212 1894 ...
1,004f33259ee4aef671c2b95d54e4be68,0 0.77 1266 583 1530 864 0 0.27 1060 582 1531 ...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1409 817 1740 1207 0 0.27 1198 818 1735...
3,009bc039326338823ca3aa84381f17f1,0 0.53 977 461 1220 762 3 0.78 661 1030 1566 1...
4,00a2145de1886cb9eb88869c85d74080,0 0.67 1123 709 1353 935 0 0.20 1167 708 1353 ...


In [23]:
print(cls.shape)
cls.head()

(3000, 2)


,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [24]:
final_pred = pd.merge(det, cls, on = 'image_id', how = 'left')
print(final_pred.shape)
final_pred.head()

(3000, 3)


,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.33 1139 631 1443 937 3 0.73 835 1212 1894 ...,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0 0.77 1266 583 1530 864 0 0.27 1060 582 1531 ...,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1409 817 1740 1207 0 0.27 1198 818 1735...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.53 977 461 1220 762 3 0.78 661 1030 1566 1...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.67 1123 709 1353 935 0 0.20 1167 708 1353 ...,0.654006


In [25]:
#Number of 'No Findings' before filtering
final_pred['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    7
Name: PredictionString, dtype: int64

In [26]:
def filter_class(row, low_thr=LOW, high_thr=HIGH):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [27]:
submission = final_pred.apply(filter_class, axis=1)
submission.head()

,image_id,PredictionString,target
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0.013326
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0 0.85 1409 817 1740 1207 0 0.27 1198 818 1735...,0.939700
3,009bc039326338823ca3aa84381f17f1,0 0.53 977 461 1220 762 3 0.78 661 1030 1566 1...,0.123799
4,00a2145de1886cb9eb88869c85d74080,0 0.67 1123 709 1353 935 0 0.20 1167 708 1353 ...,0.654006


In [28]:
#Number of 'No Findings' after filtering
submission['PredictionString'].value_counts().iloc[[0]]

14 1 0 0 1 1    1912
Name: PredictionString, dtype: int64

In [29]:
submission[['image_id', 'PredictionString']].to_csv('../checkpoint/submission.csv',index = False)